In [15]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from numpy import mean
from numpy import std
import tensorflow as tf
from matplotlib import pyplot as plt
from sklearn.model_selection import KFold
from keras.datasets import mnist
from tensorflow.keras.layers import Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import Adagrad

In [30]:
train = pd.read_csv("../input/digit-recognizer/train.csv")
test = pd.read_csv("../input/digit-recognizer/test.csv")

trainx = train.drop(columns=['label'])
trainy = train['label']

train_X, valid_X, train_y, valid_y = train_test_split(trainx, trainy, test_size=0.2, shuffle= True)


train_X = np.array(train_X).reshape(-1,28,28,1)
valid_X = np.array(valid_X).reshape(-1,28,28,1)
test = np.array(test).reshape(-1,28,28,1)

train_y = np.array(train_y)
valid_y = np.array(valid_y)

train_y = to_categorical(train_y, 10)
valid_y = to_categorical(valid_y, 10)

train_X = train_X/255.0
valid_X = valid_X/255.0
test = test/255.0

Parameters

In [24]:
parametersList = [{
    "convLayers": 2,
    "filtersNb":[64, 128],
    "filters":[(5,5), (5,5)],
    "activation": 'relu',
    "padding": 'valid',
    "fcNb": 2,
    "fcNeurons": [128, 100],
    "fcAct": 'selu',
    "dropout": [True]*3,
    "dropoutRate": [0.2]*3,
    "optimizer": 'SGD',
    "learningRate": 0.03,
    "momentum":0.9,
    "callbacks" : []
},]

CNN Model

In [31]:
m = 0
model = Sequential()
convLayers = parametersList[m]["convLayers"]
fcLayers = parametersList[m]["fcNb"]
movingShape= np.array([28,28])
for i in range(convLayers):
    print(movingShape)
    model.add(Conv2D(parametersList[m]["filtersNb"][i], parametersList[m]["filters"][i], activation=parametersList[m]["activation"], input_shape=movingShape.tolist() + [1], padding =parametersList[m]["padding"] ))
    model.add(MaxPooling2D((2, 2)))
    movingShape = np.floor((movingShape- parametersList[m]["filters"][i]  +1)/2)
model.add(Flatten())

for i in range(fcLayers):
    if parametersList[m]["dropout"][i]:
        model.add(Dropout(parametersList[m]["dropoutRate"][i]))
    model.add(Dense(parametersList[m]["fcNeurons"][i], activation=parametersList[m]["fcAct"]))

model.add(Dense(10, activation='softmax'))

# Compiling the model
if parametersList[m]["optimizer"] == 'SGD':
  opt = SGD(learning_rate=parametersList[m]["learningRate"], momentum=parametersList[m]["momentum"])
elif parametersList[m]["optimizer"] == 'adam':
  opt = Adam(learning_rate=parametersList[m]["learningRate"])

model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

# Fitting the model
model.fit(train_X, train_y, batch_size = 256,epochs=30, validation_data=(valid_X,valid_y))


[28 28]
[12. 12.]
Epoch 1/30
132/132 [==============================] - 34s 251ms/step - loss: 0.3860 - accuracy: 0.8793 - val_loss: 0.0779 - val_accuracy: 0.9742
Epoch 2/30
132/132 [==============================] - 33s 247ms/step - loss: 0.0835 - accuracy: 0.9754 - val_loss: 0.0534 - val_accuracy: 0.9830
Epoch 3/30
132/132 [==============================] - 33s 249ms/step - loss: 0.0607 - accuracy: 0.9810 - val_loss: 0.0409 - val_accuracy: 0.9863
Epoch 4/30
132/132 [==============================] - 33s 247ms/step - loss: 0.0487 - accuracy: 0.9850 - val_loss: 0.0376 - val_accuracy: 0.9882
Epoch 5/30
132/132 [==============================] - 33s 247ms/step - loss: 0.0397 - accuracy: 0.9878 - val_loss: 0.0337 - val_accuracy: 0.9887
Epoch 6/30
132/132 [==============================] - 33s 247ms/step - loss: 0.0345 - accuracy: 0.9895 - val_loss: 0.0343 - val_accuracy: 0.9885
Epoch 7/30
132/132 [==============================] - 33s 247ms/step - loss: 0.0301 - accuracy: 0.9904 - val_los

In [ ]:
preds = model.predict(test)
ImageId = np.arange(1,(test.shape[0] + 1))

my_submission = pd.DataFrame([])
my_submission['ImageId'] = ImageId
my_submission.loc[:, "label"] = np.argmax(preds, axis = 1)
my_submission.to_csv('submission.csv', index=False)